In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
PATH = "/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/train "
TEST_PATH = "/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/test"

In [ ]:
category_df = pd.read_csv(os.path.join(PATH, "category.csv"))

In [ ]:
category_df.rename(columns={'Unnamed: 0': 'Label'}, inplace=True)
category_df.head(5)

,Label,Category
0,0,Art Carney
1,1,Akemi Darenogare
2,2,Billy Bob Thornton
3,3,Bill Daley
4,4,Amanda Bynes


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import torch
from PIL import Image
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import torch.nn.functional as F
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import time
import random
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, roc_auc_score, confusion_matrix
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [ ]:
class TestDataset(Dataset):
    def __init__(self, ids, transform):
        self.ids = ids
        self.transform = transform
        
    def __getitem__(self, idx):
        name = self.ids[idx]
        img = Image.open(f'{name}')
        img = img.convert("RGB")
        img = np.array(img)
        img = img.astype("float32")
        img /= 255.

        transformed = self.transform(image=img)
        img = transformed['image']
        img = img.transpose(2,0,1).astype('float32')
        return img

    def __len__(self):
        return len(self.ids)

In [ ]:

def val_transform():
    return A.Compose([
        A.Resize(256,256,always_apply=True),
    ],p=1.)

In [ ]:
import os
dir_list = os.listdir(TEST_PATH)
names = [os.path.join(TEST_PATH, f"{count}.jpg") for count in range(len(dir_list))]

In [ ]:
names[:5]

['/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/test/0.jpg',
 '/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/test/1.jpg',
 '/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/test/2.jpg',
 '/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/test/3.jpg',
 '/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/test/4.jpg']

In [ ]:
Test_ds = TestDataset(names, val_transform)
test_loader = DataLoader(Test_ds, batch_size=64, pin_memory=True, shuffle=False, num_workers=4)

In [ ]:
data = next(iter(test_loader))
data.shape

torch.Size([64, 3, 256, 256])

In [ ]:
import torchvision.models as models

class ImageRecgoModel(nn.Module):
    def __init__(self, n_labels):
        super(ImageRecgoModel, self).__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 100)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
from torch.serialization import MAP_LOCATION
latest_path = "/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/train /Models/Best_model.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImageRecgoModel(n_labels=100).to(device)

model.load_state_dict(torch.load(latest_path,map_location=device))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 257MB/s]


<All keys matched successfully>

In [ ]:
targets = []
for X in test_loader:
  output = model(X)
  targets.append(torch.argmax(output, dim=1))


In [ ]:
number_targets = []
for i in targets:
  num = i.tolist()
  for number in num:
    number_targets.append(number)
  #name = category_df.iloc[num, 1]
  #names.append(name)

In [ ]:
names = []
for i in number_targets:
  name = category_df.iloc[i, 1]
  names.append(name)

In [ ]:
len(names)

4977

In [ ]:
df = pd.DataFrame()
df["Category"] = names
ids = [count for count in range(len(dir_list))]
df["Id"] = ids
df.set_index("Id")


,Category
Id,
0,Analeigh Tipton
1,Amy Ryan
2,Adrien Brody
3,Adriana Barraza
4,Albert Brooks
...,...
4972,Barabara Palvin
4973,Anna Faris
4974,Al Pacino


In [ ]:
SAVE_CSV_PATH = "/content/drive/MyDrive/Real Douments/MS Purdue/Spring 23/ML/submission-2.csv"
df.to_csv(SAVE_CSV_PATH, index=False)